In [1]:
import pandas as pd
import os
currentDir = os.getcwd()
print("currentDir:", currentDir)

objects_path = "StartupData/objects.csv"

objects_df = pd.read_csv(objects_path)

print(objects_df.head())
print(objects_df.columns)

currentDir: /home/linus/WashUCoding/DIS Copenhagen/Data Science/FinalProject


/tmp/ipykernel_26836/3164149082.py:8: DtypeWarning: Columns (3,7,9,10,17,18,21,22,23,25,26,29,30,37) have mixed types. Specify dtype option on import or set low_memory=False.
  objects_df = pd.read_csv(objects_path)


        id entity_type  entity_id parent_id                name  \
0      c:1     Company          1       NaN            Wetpaint   
1     c:10     Company         10       NaN             Flektor   
2    c:100     Company        100       NaN               There   
3  c:10000     Company      10000       NaN             MYWEBBO   
4  c:10001     Company      10001       NaN  THE Movie Streamer   

      normalized_name                    permalink    category_code  \
0            wetpaint            /company/wetpaint              web   
1             flektor             /company/flektor      games_video   
2               there               /company/there      games_video   
3             mywebbo             /company/mywebbo  network_hosting   
4  the movie streamer  /company/the-movie-streamer      games_video   

      status  founded_at  ... last_funding_at funding_rounds  \
0  operating  2005-10-17  ...      2008-05-19              3   
1   acquired         NaN  ...             

In [2]:
# generate a series of dictionaries, each with only rows of their own type
# there are 4 types: companies, financial orgs, people, and products
id_to_name_companies = {}
name_to_id_companies = {}

id_to_name_financial_orgs = {}
name_to_id_financial_orgs = {}

id_to_name_people = {}
name_to_id_people = {}

id_to_name_products = {}
name_to_id_products = {}

for index, row in objects_df.iterrows(): 
    id = row["id"]
    name = row["normalized_name"]

    # c: companies
    if id.startswith("c:"): 
        id_to_name_companies[id] = name
        name_to_id_companies[name] = id

    # f: financial orgs
    elif id.startswith("f:"): 
        id_to_name_financial_orgs[id] = name
        name_to_id_financial_orgs[name] = id
    
    # p: people
    elif id.startswith("p:"): 
        id_to_name_people[id] = name
        name_to_id_people[name] = id
    
    # r: products
    elif id.startswith("r:"): 
        id_to_name_products[id] = name
        name_to_id_products[name] = id

In [3]:
# Example products
print(objects_df["entity_type"].unique())
printedCount = 0
for index, row in objects_df.iterrows(): 
    if printedCount > 10: 
        break 
    if row["entity_type"] == "Product": 
        print(row["normalized_name"])
        printedCount += 1

['Company' 'FinancialOrg' 'Person' 'Product']
wikison wetpaint
slacker web player
vemo enterprise workforce planning
dexmo study and revision network
virtual pbx parachute
virtual pbx soho
virtual pbx small business
shared hosting
vps hosting
managed services
hirebridge recruiter


In [4]:
# returns a sorted list of tuples, tuple[0] is an integer code, tuple[1] is a name
def generateSortedLists(dictionary): 
    keyVals = []
    prefix = None

    # create a list of tuples without the prefix to the code
    for key, val in dictionary.items(): 
        prefix = key[:2]
        integerKey = int(key[2:]) # loses "c:" or "r:" or "f:"
        keyVals.append((integerKey,  val))

    # sort by numerical code
    keyVals.sort(key=lambda a: a[0])

    # add the prefix back to tuple[0] (the code for an element)
    return list(map(lambda x : (prefix+str(x[0]), x[1]), keyVals))

companyTuples = generateSortedLists(id_to_name_companies)
financialOrgTuples = generateSortedLists(id_to_name_financial_orgs)
peopleTuples = generateSortedLists(id_to_name_people)
productTuples = generateSortedLists(id_to_name_products)

print(companyTuples[:10])
print(financialOrgTuples[:10])
print(peopleTuples[:10])
print(productTuples[:10])

[('c:1', 'wetpaint'), ('c:2', 'adventnet'), ('c:3', 'zoho'), ('c:4', 'digg'), ('c:5', 'facebook'), ('c:7', 'omnidrive'), ('c:8', 'postini'), ('c:9', 'geni'), ('c:10', 'flektor'), ('c:11', 'fox interactive media')]
[('f:1', 'greylock'), ('f:2', 'omidyar network'), ('f:3', 'trinity ventures'), ('f:4', 'accel'), ('f:5', 'meritechpartners'), ('f:6', 'founders'), ('f:7', 'charles river ventures'), ('f:8', 'centennial ventures'), ('f:10', 'mission ventures'), ('f:11', 'sevin rosen funds')]
[('p:2', 'ben elowitz'), ('p:3', 'kevin flaherty'), ('p:4', 'raju vegesna'), ('p:5', 'ian wenig'), ('p:6', 'kevin rose'), ('p:7', 'jay adelson'), ('p:8', 'owen byrne'), ('p:9', 'ron gorodetzky'), ('p:10', 'mark zuckerberg'), ('p:11', 'dustin moskovitz')]
[('r:1', 'wikison wetpaint'), ('r:3', 'photobucket'), ('r:4', 'geni'), ('r:5', 'flektor'), ('r:9', 'gizmoz'), ('r:10', 'slacker web player'), ('r:11', 'slacker desktop radio'), ('r:13', 'slacker portable player'), ('r:14', 'lala'), ('r:15', 'helio')]


In [5]:
# Let's figure out the most common acquirers
acquistions_path = "StartupData/acquisitions.csv"

acquisitions_df = pd.read_csv(acquistions_path)
acquisitions_df.head()

,id,acquisition_id,acquiring_object_id,acquired_object_id,term_code,price_amount,price_currency_code,acquired_at,source_url,source_description,created_at,updated_at
0,1,1,c:11,c:10,NaN,20000000.0,USD,2007-05-30,http://venturebeat.com/2007/05/30/fox-interact...,Fox Interactive confirms purchase of Photobuck...,2007-05-31 22:19:54,2008-05-21 19:23:44
1,2,7,c:59,c:72,cash,60000000.0,USD,2007-07-01,http://www.techcrunch.com/2007/07/02/deal-is-c...,Deal is Confirmed: Google Acquired GrandCentral,2007-07-03 08:14:50,2011-05-06 21:51:05
2,3,8,c:24,c:132,cash,280000000.0,USD,2007-05-01,http://www.techcrunch.com/2007/05/30/cbs-acqui...,CBS Acquires Europeâs Last.fm for $280 million,2007-07-12 04:19:24,2008-05-19 04:48:50
3,4,9,c:59,c:155,cash,100000000.0,USD,2007-06-01,http://techcrunch.com/2007/05/23/100-million-p...,$100 Million Payday For Feedburner  This Deal...,2007-07-13 09:52:59,2012-06-05 03:22:17
4,5,10,c:212,c:215,cash,25000000.0,USD,2007-07-01,http://blog.seattlepi.nwsource.com/venture/arc...,seatlepi.com,2007-07-20 05:29:07,2008-02-25 00:23:47


In [26]:
from collections import defaultdict
acquisitions_dict = defaultdict(list)
numKeyErrors = 0
nonNumKeyErrors = 0

for index, row in acquisitions_df.iterrows(): 
    acquiring_object_id = row["acquiring_object_id"]
    acquired_object_id = row["acquired_object_id"]
    acquisition_cost_in_millions = row["price_amount"] // 1_000_000
    # TODO: the below line is throwing an error
    try: 
        acquirerCompanyName = str(id_to_name_companies[acquiring_object_id])
        acquiredCompanyName = str(id_to_name_companies[acquired_object_id])

        acquisitions_dict[acquirerCompanyName].append((acquiredCompanyName, acquisition_cost_in_millions))
        nonNumKeyErrors += 1

    # there are 32 key errors (i.e. 32 company codes with no known associated name)
    except KeyError: 
        # print("Key error on", acquired_object_id)
        numKeyErrors += 1

print("Number of key errors:", numKeyErrors)
print("Number of non-key errors:", nonNumKeyErrors)
print()
reducedDict = {key: len(val) for key, val in acquisitions_dict.items()}
# lambda retrieves key and value
sorted_dict = dict(sorted(reducedDict.items(), key=lambda item: item[1], reverse=True))

# print the 10 companies with the most acquisitions and what they acquired
# 
acquisitionCount = 0
for key, value in sorted_dict.items(): 
    if acquisitionCount > 10: 
        break
    print(key, sorted(acquisitions_dict[key], key=lambda item: item[1], reverse=True))
    acquisitionCount += 1



Number of key errors: 47
Number of non-key errors: 9515

cisco [('cerent', 7400.0), ('tandberg', 3400.0), ('webex', 2900.0), ('starent networks', 2900.0), ('sourcefire', 2700.0), ('meraki', 1200.0), ('ironport systems', 830.0), ('pure digital technologies', 590.0), ('intucell', 475.0), ('airespace', 450.0), ('whiptail', 415.0), ('navini networks', 330.0), ('ubiquisys', 310.0), ('lightwire', 271.0), ('postpath', 215.0), ('scansafe', 183.0), ('composite software', 180.0), ('cloupia', 125.0), ('pure networks', 120.0), ('joulex', 107.0), ('tidal software', 105.0), ('securent', 100.0), ('coreoptics', 99.0), ('bni video', 99.0), ('extend media', 80.0), ('dvn', 44.0), ('metreos', 28.0), ('jabber', 0.0), ('neopath networks', 0.0), ('broadware technologies', 0.0), ('richards zeta building intelligence', 0.0), ('nuova systems', 0.0), ('fast data technology', 0.0), ('tribe', 0.0), ('fiveacross', 0.0), ('moto development group', 0.0), ('divitech a s', 0.0), ('arch rock', 0.0), ('crescendo communic